In [1]:
# %load proj_load_table.py
import numpy as np
from keras.preprocessing import image

def load_seq(filename, B, N, F, D):
    with open(filename) as file:
        text = file.read()
        text = text.splitlines()
    dat = np.zeros((len(text), 10))
    for i in range(0,len(text)):
        l = text[i].split(',')
        for j in range(0,len(l)):
            dat[i][j] = float(l[j])
    
    data = np.full((B, N, F, D), np.nan)
    label = np.full((B, N, F), np.nan, dtype=int)
    num_targets = np.zeros((B, F), dtype=int)
    i = 0
    k = 0
    while dat[i][0] <= B * F:
        b = int((dat[i][0]-1) // F)
        r = int((dat[i][0]-1) % F)
        data[b][k][r][:] = dat[i][2:6]
        label[b][k][r] = int(dat[i][1])
        i += 1
        k += 1
        while k >= N and dat[i][0] == dat[i-1][0]:
            i += 1
        if i > 0 and dat[i][0] != dat[i-1][0]:
            num_targets[b][r] = k
            k = 0
    return (data, label)
    
def loadTable(filename, B, M, N, F, D):
    (detections, dummy) = load_seq(filename + "det/det.txt", B, N, F, D)
    img = image.load_img(filename + "det/000001-acf.jpg")
    sz = img.size
    detections[0,:,:,0] = detections[0,:,:,0]/sz[0] - 0.5
    detections[0,:,:,1] = detections[0,:,:,1]/sz[1] - 0.5
    detections[0,:,:,2] = detections[0,:,:,2]/sz[0] - 0.5
    detections[0,:,:,3] = detections[0,:,:,3]/sz[1] - 0.5
    (tracks, labels) = load_seq(filename + "gt/gt.txt", B, M, F, D)
    tracks[0,:,:,0] = tracks[0,:,:,0]/sz[0] - 0.5
    tracks[0,:,:,1] = tracks[0,:,:,1]/sz[1] - 0.5
    tracks[0,:,:,2] = tracks[0,:,:,2]/sz[0] - 0.5
    tracks[0,:,:,3] = tracks[0,:,:,3]/sz[1] - 0.5
    return (tracks, detections, labels)

# Example: traindata = loadTable("/home/raj/Documents/project/amilan-rnntracking-64d477848af3/src/2DMOT2015Labels/train/ADL-Rundle-8/", 1, 20, 20, 20, 4)


Using Theano backend.


In [37]:
traindata = loadTable("/home/raj/Documents/project/amilan-rnntracking-64d477848af3/src/2DMOT2015Labels/train/ADL-Rundle-8/", B, N, M, F, D)

In [4]:
traindata[0].shape

(1, 20, 20, 4)

In [3]:
import os
import PIL
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Input, Flatten, Lambda, SimpleRNN, LSTM
from keras import backend as K
from keras.models import Model

In [4]:
def getDA(N, M, hidden_units):
    a = Input(shape=(N,M))
    b = LSTM(hidden_units, return_sequences=True, activation='tanh')(a)
    c = Dense(M, activation='softmax')(b)
    model = Model(input=a, output=c)
    return model

In [10]:
x = Input(shape=(20,4))
h = SimpleRNN(100, return_sequences=True, activation='tanh')(x)
xs = Dense(4, activation='linear')(h)
z = Input(shape=(20,4))
A = Input(shape=(20,20))
zx = concatenate((z, xs), axis=0)
az = dot((A,zx), axes=0)
E = Input(shape=(20,1))
ea = multiply((E,az))
j = Dense(20, activation='linear')(ea)
jh = add((j,h))
th = Activation(activation='tanh')(jh)
xt1 = Dense(4, activation='linear')(th)
Et1 = Dense(20, activation='sigmoid')(th)
et = Lambda(lambda x: -x, output_shape=input_shape)(E)
En = add((Et1, et))
Es = Lambda(lambda x: abs(x), output_shape=input_shape)(En)

NameError: name 'concatenate' is not defined

In [36]:
N = 3
M = 20
F = 70
B = 1
D = 4
DA = getDA(N, M, 50)

In [38]:
tracks = traindata[0]
detections = traindata[1]
labels = traindata[2]
tlabel = np.eye(M)[labels[0,:,:]-1]

In [39]:
trainlabel = np.transpose(tlabel, (1, 0, 2))

In [61]:
labels.shape

(1, 2, 30)

In [135]:
trainlabel.shape

(50, 5, 20)

In [40]:
Ct = np.full((F, N, M), 1000, dtype=float)
for t in range(0,F):
    for i in range(0,N):
        for j in range(0,M):
            dist = np.linalg.norm(tracks[0,i,t,:]-detections[0,j,t,:])
            if not np.isnan(dist) and not np.isinf(dist):
                Ct[t, i, j] = dist

In [72]:
np.nan == np.nan

False

In [41]:
DA.compile( loss="categorical_crossentropy", optimizer="Nadam", metrics = ['categorical_accuracy'])

In [45]:
history1 = DA.fit(Ct, trainlabel, batch_size=2, nb_epoch=10, validation_data=(Cttest, testlabel))

Train on 70 samples, validate on 70 samples
Epoch 1/10
70/70 [==============================] - 0s - loss: 2.9763 - categorical_accuracy: 0.0905 - val_loss: 2.3091 - val_categorical_accuracy: 0.3048
Epoch 2/10
70/70 [==============================] - 0s - loss: 2.1304 - categorical_accuracy: 0.3286 - val_loss: 1.5465 - val_categorical_accuracy: 0.3238
Epoch 3/10
70/70 [==============================] - 0s - loss: 1.5908 - catego

In [34]:
vars(history1)

{'epoch': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'history': {'categorical_accuracy': [0.90000000000000002,
   0.93000000000000005,
   0.93000000000000005,
   0.91000000000000003,
   0.89000000000000001,
   0.93999999999999995,
   0.95999999999999996,
   0.95999999999999996,
   0.92000000000000004,
   0.91000000000000003],
  'loss': [0.45758197963237762,
   0.44622089326381681,
   0.43796322822570799,
   0.43401142239570617,
   0.45352408945560457,
   0.44511506795883177,
   0.43735260486602784,
   0.42646240592002871,
   0.42270655632019044,
   0.41367502033710479],
  'val_categorical_accuracy': [0.47999999999999998,
   0.47999999999999998,
   0.47999999999999998,
   0.68999999999999995,
   0.68999999999999995,
   0.68999999999999995,
   0.68999999999999995,
   0.68999999999999995,
   0.68999999999999995,
   0.68999999999999995],
  'val_loss': [3.3869838416576385,
   3.3499550795555115,
   3.3566124713420868,
   3.2539200627803804,
   3.2435656237602233,
   3.2519214904308318,
   3.277244255

In [104]:
trainlabel[0,0,:]

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [111]:
np.isnan(Ct)

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       ..., 
       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]], dtype=bool)

In [144]:
DA.predict(Ct, batch_size=1)

array([[[  5.14513314e-01,   4.83486921e-01,   6.20137507e-05, ...,
           7.84870572e-05,   2.40030247e-04,   4.77004214e-05],
        [  3.30684274e-01,   6.68846190e-01,   1.45569211e-05, ...,
           1.57500253e-05,   6.00201347e-05,   1.01489641e-05]],

       [[  5.57938337e-01,   4.33450669e-01,   3.06920527e-04, ...,
           3.69545160e-04,   7.91433442e-04,   6.45063934e-04],
        [  4.09622431e-01,   5.87414026e-01,   9.69099056e-05, ...,
           1.23461607e-04,   2.90373515e-04,   2.26914271e-04]],

       [[  5.14510512e-01,   4.83489782e-01,   6.20132341e-05, ...,
           7.84861695e-05,   2.40029389e-04,   4.77002977e-05],
        [  3.30683857e-01,   6.68846607e-01,   1.45569020e-05, ...,
           1.57499908e-05,   6.00201165e-05,   1.01489513e-05]],

       ..., 
       [[  6.12127960e-01,   3.83218735e-01,   1.32018409e-04, ...,
           2.34506966e-04,   2.53888807e-04,   2.53700127e-04],
        [  4.37816471e-01,   5.60351610e-01,   4.38954194

In [42]:
testdata = loadTable("/home/raj/Documents/project/amilan-rnntracking-64d477848af3/src/2DMOT2015Labels/train/ADL-Rundle-6/", B, N, M, F, D)

In [43]:
testtracks = testdata[0]
testdetections = testdata[1]
labelstest = testdata[2]
tlabeltest = np.eye(M)[labelstest[0,:,:]-1]
testlabel = np.transpose(tlabeltest, (1, 0, 2))

In [44]:
Cttest = np.full((F, N, M), 1000, dtype=float)
for t in range(0,F):
    for i in range(0,N):
        for j in range(0,M):
            dist = np.linalg.norm(testtracks[0,i,t,:]-testdetections[0,j,t,:])
            if not np.isnan(dist) and not np.isinf(dist):
                Cttest[t, i, j] = dist
        

In [15]:
def getDistMat(tracks, detections, F, N, M):
    label = np.zeros((F, N, M))
    Ct = np.full((F, N, M), 1000, dtype=float)
    for t in range(0,F):
        for i in range(0,N):
            for j in range(0,M):
                dist = np.linalg.norm(tracks[0,i,t,:]-detections[0,j,t,:])
                if not np.isnan(dist) and not np.isinf(dist):
                    Ct[t, i, j] = dist
            ind = np.argmin(Ct[t,i,:])
            label[t,i,ind] = 1
    return (Ct, label)

In [181]:
(train_x, train_y) = getDistMat(tracks, detections, F, N, M)

In [182]:
(test_x, test_y) = getDistMat(testtracks, testdetections, 20, N, M)

In [198]:
DA.fit(train_x, train_y, batch_size=1, nb_epoch=30, validation_data=(test_x, test_y))

Train on 50 samples, validate on 20 samples
Epoch 1/30
50/50 [==============================] - 0s - loss: 1.2473 - categorical_accuracy: 0.4200 - val_loss: 4.8613 - val_categorical_accuracy: 0.0000e+00
Epoch 2/30
50/50 [==============================] - 0s - loss: 1.2587 - categorical_accuracy: 0.4200 - val_loss: 4.7172 - val_categorical_accuracy: 0.0000e+00

In [193]:
train_x[0,:,:]

array([[  4.75268044e-01,   3.26639801e-01,   7.01446212e-02,
          5.28695751e-01,   1.55897090e-01,   4.06409531e-02,
          3.40147299e-01,   3.04549411e-01,   3.67135850e-01,
          4.28363167e-01,   1.94473888e-01,   5.24501643e-01,
          3.10888680e-01,   3.69823887e-01,   5.72185470e-01,
          1.00000000e+03,   1.00000000e+03,   1.00000000e+03,
          1.00000000e+03,   1.00000000e+03]])

In [190]:
pred = DA.predict(train_x, batch_size=5)

In [192]:
pred[0,:,:]

array([[  1.25157088e-01,   8.81014168e-02,   3.35764527e-01,
          3.35447490e-03,   1.40277535e-01,   2.58184016e-01,
          4.05578278e-02,   1.16352248e-03,   1.47408678e-03,
          3.92952585e-04,   5.67072362e-04,   3.97325552e-04,
          1.47524988e-03,   3.88229586e-04,   6.94917166e-04,
          3.81619146e-04,   5.40560344e-04,   2.36898486e-04,
          4.49306623e-04,   4.41326119e-04]], dtype=float32)

In [194]:
train_y[0,:,:]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [196]:
tracks[0,0,:]

array([[ 0.21197917, -0.13611111, -0.46197917, -0.29166667],
       [ 0.2109375 , -0.13611111, -0.46197917, -0.28981481],
       [ 0.20989583, -0.13611111, -0.4609375 , -0.28611111],
       [ 0.20885417, -0.13611111, -0.45989583, -0.28425926],
       [ 0.20885417, -0.13611111, -0.45989583, -0.28055556],
       [ 0.2078125 , -0.13611111, -0.45989583, -0.2787037 ],
       [ 0.20677083, -0.13611111, -0.45885417, -0.275     ],
       [ 0.20572917, -0.13611111, -0.4578125 , -0.27314815],
       [ 0.20572917, -0.13425926, -0.4578125 , -0.2712963 ],
       [ 0.20677083, -0.13425926, -0.4578125 , -0.26944444],
       [ 0.2078125 , -0.13425926, -0.45677083, -0.26759259],
       [ 0.20989583, -0.13425926, -0.45677083, -0.26388889],
       [ 0.2109375 , -0.13425926, -0.45572917, -0.26203704],
       [ 0.21302083, -0.13425926, -0.45572917, -0.25833333],
       [ 0.2140625 , -0.13425926, -0.4546875 , -0.25648148],
       [ 0.21614583, -0.13425926, -0.4546875 , -0.25277778],
       [ 0.2140625 , -0.